In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install fastapi uvicorn numpy nest_asyncio pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.1 MB/s eta 0:00:00


In [3]:
!ngrok config add-authtoken 2ukYFbl5XHexX3hDsOpYSnABfex_5ApsW6UsEqz23HpEUVpVx

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import uvicorn
from fastapi import FastAPI, WebSocket
import pandas as pd
import numpy as np
import asyncio
import nest_asyncio
from pyngrok import ngrok
from scipy.signal import butter, filtfilt, find_peaks
from fastapi.middleware.cors import CORSMiddleware  # CORS middleware

nest_asyncio.apply()

# Load ECG dataset
file_path = "/content/drive/MyDrive/Mangesh/Patient-Idcsv.csv"
df = pd.read_csv(file_path)

# Bandpass Filter Function
def bandpass_filter(data, lowcut=0.5, highcut=50.0, fs=250, order=2):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

# Normalize function
def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

# Extract and process ECG data
def get_ecg_data():
    fs = 250  # Sampling frequency
    for _, row in df.iterrows():
        ecg_signal = np.array(list(map(int, row['ValueStr'].split(','))))  # Convert string to list of integers
        filtered_ecg = bandpass_filter(ecg_signal, lowcut=0.5, highcut=50.0, fs=fs)
        is_inverted = np.max(filtered_ecg) < abs(np.min(filtered_ecg))
        processed_ecg = -filtered_ecg if is_inverted else filtered_ecg
        normalized_ecg = normalize(processed_ecg)
        r_peaks, _ = find_peaks(normalized_ecg, distance=fs//2, height=np.mean(normalized_ecg) + np.std(normalized_ecg))

        # Generate response
        yield {
            "time": list(range(len(normalized_ecg))),
            "values": normalized_ecg.tolist(),
            "r_peaks": r_peaks.tolist(),
        }

# FastAPI Setup
app = FastAPI()

# Enable CORS for WebSocket connections
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow all origins
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

async def send_ecg_data(websocket: WebSocket):
    await websocket.accept()
    print("Client connected!")  # Debugging log
    try:
        for data in get_ecg_data():
            await websocket.send_json(data)
            await asyncio.sleep(0.5)  # Simulate real-time data feed
    except Exception as e:
        print(f"Error: {e}")  # Print error if any
    finally:
        print("Client disconnected!")  # Debugging log

@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await send_ecg_data(websocket)

# Start FastAPI server
port = 8000
public_url = ngrok.connect(port).public_url
print(f"Public WebSocket URL: {public_url}/ws")  # Copy this URL and use in index.html

uvicorn.run(app, host="0.0.0.0", port=port)


INFO:     Started server process [945]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public WebSocket URL: https://9bdd-34-16-147-6.ngrok-free.app/ws


INFO:     ('202.179.92.106', 0) - "WebSocket /ws" [accepted]
INFO:     connection open


Client connected!
